In [68]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [69]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [70]:
## Read Data for the Fraudulent Email Kaggle Challenge
data_train = pd.read_csv("/Users/feepieper/Library/CloudStorage/OneDrive-Persönlich/Ironhack/Module4/labs/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')
data_test = pd.read_csv("/Users/feepieper/Library/CloudStorage/OneDrive-Persönlich/Ironhack/Module4/labs/lab-natural-language-processing/data/kg_test.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data_train = data_train.head(1000)
print(data_train.shape)
data_train.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [71]:
# Your code
from sklearn.model_selection import train_test_split

train_set, val_set = train_test_split(data_train, test_size=0.2, random_state=42)
print(train_set.shape, val_set.shape)

(800, 2) (200, 2)


## Data Preprocessing

In [72]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [73]:
# Your code
def clean_text(data):
    # 1️⃣ Remove inline JavaScript and CSS
    data = re.sub(r'<script.*?>.*?</script>', '', data, flags=re.DOTALL | re.IGNORECASE)
    data = re.sub(r'<style.*?>.*?</style>', '', data, flags=re.DOTALL | re.IGNORECASE)

    # 2️⃣ Remove HTML comments
    data = re.sub(r'<!--.*?-->', '', data, flags=re.DOTALL)

    # 3️⃣ Remove remaining HTML tags
    data = re.sub(r'<[^>]+>', '', data)

    # Optional: remove extra whitespace
    data = re.sub(r'\s+', ' ', data).strip()

    return data

train_set['preprocessed_text'] = train_set['text'].apply(clean_text)
val_set['preprocessed_text'] = val_set['text'].apply(clean_text)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [74]:
# Your code
def further_clean_text(text):
    # 1️⃣ Remove all special characters (keep letters and spaces)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # 2️⃣ Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # 3️⃣ Remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # 4️⃣ Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    
    # 5️⃣ Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    
    # 6️⃣ Remove prefixed 'b' (like b'example')
    text = re.sub(r"^b\s+", '', text)
    
    # 7️⃣ Convert to lowercase
    text = text.lower()
    
    # Optional: strip leading/trailing spaces
    text = text.strip()
    
    return text

In [75]:
train_set['preprocessed_text'] = train_set['preprocessed_text'].apply(further_clean_text)
val_set['preprocessed_text'] = val_set['preprocessed_text'].apply(further_clean_text)

## Now let's work on removing stopwords
Remove the stopwords.

In [76]:
# Your code
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords_only(text):
    # Text in Wörter aufteilen
    words = text.split()
    # Stopwords entfernen
    words = [word for word in words if word not in stop_words]
    # Wieder zu String zusammenfügen
    return ' '.join(words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/feepieper/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
train_set['preprocessed_text'] = train_set['preprocessed_text'].apply(remove_stopwords_only)
val_set['preprocessed_text'] = val_set['preprocessed_text'].apply(remove_stopwords_only)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [78]:
# Your code
import nltk
from nltk.stem import WordNetLemmatizer

# Sicherstellen, dass WordNet-Daten vorhanden sind
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Text in Wörter aufteilen
    words = text.split()
    # Wörter lemmatisieren
    words = [lemmatizer.lemmatize(word) for word in words]
    # Wieder zu String zusammenfügen
    return ' '.join(words)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/feepieper/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/feepieper/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [79]:
train_set['preprocessed_text'] = train_set['preprocessed_text'].apply(lemmatize_text)
val_set['preprocessed_text'] = val_set['preprocessed_text'].apply(lemmatize_text)

In [80]:
train_set.head()

,text,label,preprocessed_text
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,regard mr nelson smith kindly reply private em...
535,I have not been able to reach oscar this am. W...,0,able reach oscar supposed send pdb receive
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,huma abedin checking pat work jack jake rest a...
557,I can have it announced here on Monday - can't...,0,announced monday today
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,bank africaagence san pedro bp san pedro cote ...


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [82]:
# Your code
def get_top_n_words(df, label, n=10):
    # Filter für die Klasse (ham/spam)
    texts = df[df['label'] == label]['preprocessed_text']
    
    # Bag-of-Words
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(texts)
    
    # Summiere die Häufigkeiten
    word_counts = X.sum(axis=0)
    word_freq = [(word, word_counts[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    
    # Sortieren nach Häufigkeit
    word_freq = sorted(word_freq, key=lambda x: x[1], reverse=True)
    
    # Top N zurückgeben
    return word_freq[:n]

top_ham = get_top_n_words(train_set, 0, n=10)
print("Top 10 Ham Words:", top_ham)

# Top 10 Wörter in Spam (label 1)
top_spam = get_top_n_words(train_set, 1, n=10)
print("Top 10 Spam Words:", top_spam)

Top 10 Ham Words: [('state', 117), ('pm', 97), ('would', 94), ('president', 89), ('mr', 89), ('time', 81), ('percent', 80), ('obama', 77), ('call', 74), ('secretary', 74)]
Top 10 Spam Words: [('money', 842), ('account', 740), ('bank', 645), ('fund', 625), ('transaction', 466), ('business', 424), ('mr', 422), ('country', 419), ('million', 369), ('company', 365)]


## Extra features

In [86]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

train_set['money_mark'] = train_set['preprocessed_text'].str.contains(money_simbol_list)*1
train_set['suspicious_words'] = train_set['preprocessed_text'].str.contains(suspicious_words)*1
train_set['text_len'] = train_set['preprocessed_text'].apply(lambda x: len(x)) 

val_set['money_mark'] = val_set['preprocessed_text'].str.contains(money_simbol_list)*1
val_set['suspicious_words'] = val_set['preprocessed_text'].str.contains(suspicious_words)*1
val_set['text_len'] = val_set['preprocessed_text'].apply(lambda x: len(x)) 

train_set.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,regard mr nelson smith kindly reply private em...,0,0,79
535,I have not been able to reach oscar this am. W...,0,able reach oscar supposed send pdb receive,0,0,42
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,huma abedin checking pat work jack jake rest a...,0,0,76
557,I can have it announced here on Monday - can't...,0,announced monday today,0,0,22
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,bank africaagence san pedro bp san pedro cote ...,1,1,1052


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [87]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# 1️⃣ Load the vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # optional: max_features, stop_words='english'

# 2️⃣ Vectorize all dataset (hier z. B. clean_text_lemma Spalte)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_set['preprocessed_text'])
X_val_tfidf = tfidf_vectorizer.transform(val_set['preprocessed_text'])

# 3️⃣ Print the shape of the vectorized dataset
print("Train TF-IDF shape:", X_train_tfidf.shape)
print("Validation TF-IDF shape:", X_val_tfidf.shape)


Train TF-IDF shape: (800, 5000)
Validation TF-IDF shape: (200, 5000)


## And the Train a Classifier?

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# 1️⃣ TF-IDF Vektorisierung
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # optional: stop_words='english'
X_train_tfidf = tfidf_vectorizer.fit_transform(train_set['preprocessed_text'])
X_val_tfidf = tfidf_vectorizer.transform(val_set['preprocessed_text'])

# 2️⃣ Multinomial Naive Bayes Classifier
nb_clf = MultinomialNB()
nb_clf.fit(X_train_tfidf, train_set['label'])

# 3️⃣ Vorhersage auf Validation Set
y_pred = nb_clf.predict(X_val_tfidf)

# 4️⃣ Evaluation
print("Accuracy:", accuracy_score(val_set['label'], y_pred))
print("\nClassification Report:\n", classification_report(val_set['label'], y_pred))


Accuracy: 0.975

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98       125
           1       0.94      1.00      0.97        75

    accuracy                           0.97       200
   macro avg       0.97      0.98      0.97       200
weighted avg       0.98      0.97      0.98       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code